In [25]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time

In [26]:
train = pd.read_csv('../../Paper 1/UNSW-NB15/Dataset UNSW15/UNSW_NB15_training-set.csv')
test = pd.read_csv('../../Paper 1/UNSW-NB15/Dataset UNSW15/UNSW_NB15_testing-set.csv')

In [27]:
#drop all rows with attack category as backdoor,shellcode,worms and dos
train = train[train.attack_cat != 'Backdoor']
train = train[train.attack_cat != 'Shellcode']
train = train[train.attack_cat != 'Worms']
train = train[train.attack_cat != 'DoS']
test = test[test.attack_cat != 'Backdoor']
test = test[test.attack_cat != 'Shellcode']
test = test[test.attack_cat != 'Worms']
test = test[test.attack_cat != 'DoS']

### Data Pre-processing

In [28]:
# combine train and test data
data = pd.concat([train,test]).reset_index(drop=True)

# Splitting the data into categorical and numerical columns
cols_cat = data.select_dtypes('object').columns 
cols_numeric = data._get_numeric_data().columns

In [29]:
data['label'].value_counts()

label
1    144306
0     93000
Name: count, dtype: int64

In [30]:
def Remove_dump_values(data, cols):
    for col in cols:
        # replace all values with '-' to 'None'
        data[col] = np.where(data[col] == '-', 'None', data[col])
    return data

In [31]:
cols = data.columns
# Remove the dump values
data_bin = Remove_dump_values(data, cols)
# remvoe the id column
data_bin = data_bin.drop(['id'], axis=1)
data_bin_attackcat = data_bin[['attack_cat']]

In [32]:
# ordinal encode attack category
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories='auto')
oe.fit(data_bin_attackcat)
data_bin_attackcat = oe.transform(data_bin_attackcat)

In [33]:
data_bin.drop(['attack_cat'], axis=1, inplace=True)
cols_cat = cols_cat.drop(['attack_cat'])

#  one hot encode the categorical columns and concat with the numerical columns
data_bin_hot = pd.get_dummies(data_bin,columns=cols_cat)
attack_cat_df = pd.DataFrame(data_bin_attackcat, columns=['attack_cat'])
data_bin_hot2 = pd.concat([data_bin_hot,attack_cat_df],axis=1, join='inner',ignore_index=False)

In [34]:
cols_numeric = list(cols_numeric)
cols_numeric.remove('label')
cols_numeric.remove('id')

In [35]:
# convert the numerical columns to float and normalize the data
data_bin_hot2[cols_numeric] = data_bin_hot2[cols_numeric].astype('float') 
data_bin_hot2[cols_numeric] = (data_bin_hot2[cols_numeric] - np.min(data_bin_hot2[cols_numeric])) / np.std(data_bin_hot2[cols_numeric])

In [36]:
data_bin_hot2['attack_cat'].value_counts() 

attack_cat
4.0    93000
3.0    58871
1.0    44525
2.0    24246
5.0    13987
0.0     2677
Name: count, dtype: int64

In [37]:
# oversample minority types, which are all attack types other then normal using SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X = data_bin_hot2.drop('attack_cat', axis=1)
y = data_bin_hot2['attack_cat']
X_res, y_res = sm.fit_resample(X, y)

# convert to dataframe
X_res = pd.DataFrame(X_res, columns=X.columns)
y_res = pd.DataFrame(y_res, columns=['attack_cat'])

# concatenate X_res and y_res
data_balanced = pd.concat([X_res, y_res], axis=1)

# check new class counts
data_balanced['attack_cat'].value_counts()

attack_cat
4.0    93000
5.0    93000
1.0    93000
0.0    93000
2.0    93000
3.0    93000
Name: count, dtype: int64

In [38]:
# save the balanced data
data_balanced.to_csv('UNSW_NB15_balanced_smote.csv', index=False)